# Mid Term Exam 

## Shuvrangshu Mukhopadhyay

### Chat Extension Cassandra Database

#### Installing required packages for running the python script

In [ ]:
pip install pandas
pip install cassandra-driver

#### Importing the necessary packages

In [5]:
import pandas as pd
from cassandra.cluster import Cluster

Defaulting to user installation because normal site-packages is not writeable
  Using cached cassandra_driver-3.28.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (19.1 MB)
  Using cached geomet-0.2.1.post1-py3-none-any.whl (18 kB)
Note: you may need to restart the kernel to use updated packages.


### Creating a connection to the cassandra cluster which we have started in the terminal using 'Docker-Compose up'

In [7]:
clstr = Cluster()
session = clstr.connect()

### We would want to create a schema, which is also known as the keyspace in Cassandra, but first check if we have the already or not?

In [77]:
session.execute("DROP KEYSPACE IF EXISTS chatdata")    # Checking and dropping chatData keyspace if exists

#### It does not exist, let create our own keyspace. We will create a keyspace with replication factor 3 and name as chatdata. Since we are trying to store chat data, we would want to create chat data

In [78]:
# Keeping replication factor as 3, as it must not exceed more than the number of nodes present (We have 3 here)
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS chatData WITH REPLICATION = {
        'class':'SimpleStrategy', 
        'replication_factor':3                                               
    }
""")

#### Let's see if we the keyspace created or not?

In [79]:
rows = session.execute("desc keyspaces")
for row in rows:
    print(f"{row[0]}")

chatdata
system
system_auth
system_distributed
system_schema
system_traces
system_views
system_virtual_schema


In [80]:
session.execute("desc keyspaces")

#### We can see our keyspace created.

## Now in the below cell, we will create our table with 4 columns. Since user-id would be unique to each users, it is a good choice for our primary key.

In [81]:
session.execute("""
CREATE TABLE IF NOT EXISTS chatdata.chatsession ( 
    user_id INT, 
    chatdate TEXT, 
    file_type TEXT, 
    file TEXT,
    PRIMARY KEY(user_id)
);
""")

In [90]:
testfile = pd.read_csv('testfile.csv')

In [91]:
testfile

,user_id,chat_date,file,file_type
0,101,'10/04/2023','This is a random file for testing purpose','File'


### Below code is pushing 1 record to the thable which we created in the previous steps

In [92]:
for index, row in testfile.iterrows():
    print(f"""
        INSERT INTO chatdata.chatsession (user_id, chatdate, file, file_type)     
        VALUES ({row.user_id},{row.chat_date}, {row.file}, {row.file_type});
        """)
    session.execute(f"""
        INSERT INTO chatdata.chatsession (user_id, chatdate, file, file_type)     
        VALUES ({row.user_id},{row.chat_date}, {row.file}, {row.file_type});
        """)


        INSERT INTO chatdata.chatsession (user_id, chatdate, file, file_type)     
        VALUES (101,'10/04/2023', 'This is a random file for testing purpose', 'File');
        


### Below is another way to insert records based on a text file:

In [93]:
with open('testfile.txt', 'r') as inp:
    temp = inp.read()

In [94]:
temp

'This is another way of recording text and we will use this to store the date'

In [95]:
session.execute(f"""
        INSERT INTO chatdata.chatsession (user_id,chatdate,file,file_type)     
        VALUES (101,"10052023",{temp},"File");
        """)


## For some reason this way is not working, however, the csql table has 2 records for now

SyntaxException: <Error from server: code=2000 [Syntax error in CQL query] message="line 3:30 no viable alternative at input ',' (...file,file_type)             VALUES (101,["1005202]3",...)">

### Below snippet of the code can be utilised to add more records to the existing table:

In [96]:
session.execute(f"""
        INSERT INTO chatdata.chatsession (user_id, chatdate, file, file_type)     
        VALUES (102,'10/05/2023','Text used in the attachment file','File');
        """)

### Below snippet of the code can be used for searching a record based on any parameter, I am using user_id here

In [97]:
result = session.execute(f"""
        SELECT * FROM chatdata.chatsession WHERE user_id = 101;
        """)

for items in result:
    print(items)

Row(user_id=101, chatdate='10/04/2023', file='This is a random file for testing purpose', file_type='File')


### Below snippet of the code can be used to delete a record based on any parameter, I am using user_id here

In [98]:
session.execute("""
        DELETE FROM chatdata.chatsession WHERE user_id = 102;
        """)



In [99]:
after_delete = session.execute('''SELECT * FROM chatdata.chatsession''')

for items in after_delete:
    print(items)

Row(user_id=101, chatdate='10/04/2023', file='This is a random file for testing purpose', file_type='File')


### We can see in the top output that user_id 102' record has been deleted